In [7]:
from stable_baselines3.common.callbacks import BaseCallback

class CustomProgressCallback(BaseCallback):
    def __init__(self, check_freq, verbose=1):
        super(CustomProgressCallback, self).__init__(verbose)
        self.check_freq = check_freq

    def _on_step(self) -> bool:
        if self.n_calls % self.check_freq == 0:
            mean_reward = np.mean(self.locals["rewards"])
            print(f"Step: {self.num_timesteps} | Mean Reward: {mean_reward:.2f}")
        return True

In [13]:
import gymnasium as gym
from stable_baselines3 import PPO
from ale_py import ALEInterface
ale = ALEInterface()
import ale_py

# Create the Donkey Kong environment
# The environment id is typically "ALE/DonkeyKong-v5"; adjust if necessary.
env = gym.make("ALE/DonkeyKong-v5", render_mode="rgb_array", frameskip=4)
# env = ResizeObservation(env, shape=84)
model = PPO("CnnPolicy", env, n_steps=512, batch_size=64, device='auto', verbose=1, tensorboard_log="./ppo_donkeykong_log/")
env = RecordVideo(env, video_folder="./videos", episode_trigger=lambda x: x % 10 == 0)

# Create a PPO agent with a CNN policy (good for image-based inputs)
model = PPO("CnnPolicy", env, verbose=1)

# Train the agent for 100,000 timesteps (adjust timesteps as needed)
model.learn(total_timesteps=100000)

# Save the trained model
model.save("ppo_donkeykong")
print("Training complete and model saved as 'ppo_donkeykong'.")

# OPTIONAL: Demonstrate the trained agent
obs, info = env.reset()
done = False
while not done:
    action, _ = model.predict(obs)
    obs, reward, done, truncated, info = env.step(action)
    # For debugging, you might want to display or process the frame.
env.close()


Using cuda device
Wrapping the env with a `Monitor` wrapper
Wrapping the env in a DummyVecEnv.
Wrapping the env in a VecTransposeImage.
Using cuda device
Wrapping the env with a `Monitor` wrapper
Wrapping the env in a DummyVecEnv.
Wrapping the env in a VecTransposeImage.
---------------------------------
| rollout/           |          |
|    ep_len_mean     | 803      |
|    ep_rew_mean     | 50       |
| time/              |          |
|    fps             | 236      |
|    iterations      | 1        |
|    time_elapsed    | 8        |
|    total_timesteps | 2048     |
---------------------------------
-----------------------------------------
| rollout/                |             |
|    ep_len_mean          | 776         |
|    ep_rew_mean          | 75          |
| time/                   |             |
|    fps                  | 135         |
|    iterations           | 2           |
|    time_elapsed         | 30          |
|    total_timesteps      | 4096        |
| train/  

In [ ]:
import matplotlib.pyplot as plt
import pandas as pd

# Load TensorBoard log data
log_data = pd.read_csv("./ppo_donkeykong_log/PPO_1/events.out.tfevents.*", sep="\t")

plt.plot(log_data['Step'], log_data['Value'], label='Episode Reward')
plt.title("Training Progress")
plt.xlabel("Timesteps")
plt.ylabel("Reward")
plt.legend()
plt.grid(True)
plt.show()
